In [1]:
# from https://colab.research.google.com/drive/1uL1TdMbR4kqa0Ksrd_Of_jWSxWt1ia7o?usp=sharing#scrollTo=c48a272c-8e87-4740-9960-129d7d5943bb
# https://betterprogramming.pub/llamaindex-deep-lake-for-financial-statement-analysis-954f2b789c8e


In [2]:
import sys
from datetime import datetime
from IPython.display import Markdown, display
from pathlib import Path

from llama_index import download_loader, ServiceContext, StorageContext, load_index_from_storage, GPTVectorStoreIndex
from llama_index import GPTListIndex, LLMPredictor
from llama_index.composability import ComposableGraph

from langchain import OpenAI

import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import dotenv
dotenv.load_dotenv()


True

In [3]:
# # set text wrapping (for colab?)
# from IPython.display import HTML, display

# def set_css():
#   display(HTML('''
#   <style>
#     pre {
#         white-space: pre-wrap;
#     }
#   </style>
#   '''))
# get_ipython().events.register('pre_run_cell', set_css)

## Ingest data

In [4]:
# mkdir uber
# mkdir tmp
# wget https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1 -O tmp/UBER.zip
# unzip tmp/UBER.zip -d tmp
# mv tmp/UBER/*.html ./uber

In [5]:
# extract raw text from html
# https://unstructured.io ; https://github.com/Unstructured-IO/unstructured
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)

loader = UnstructuredReader()
doc_set = {}
all_docs = []
years = [2022, 2021, 2020, 2019]
for year in years:
    year_docs = loader.load_data(file=Path(f'./uber/UBER_{year}.html'), split_documents=False)
    # insert year metadata into each year
    for d in year_docs:
        d.extra_info = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)
    

[nltk_data] Downloading package punkt to /Users/drucev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/drucev/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:unstructured:Reading document from string ...


Reading document from string ...


INFO:unstructured:Reading document ...


Reading document ...


INFO:unstructured:Reading document from string ...


...

Reading document ...


INFO:unstructured:Reading document from string ...


Reading document from string ...


INFO:unstructured:Reading document ...


Reading document ...


In [6]:
# service_context talks to openai (or other llm)
# https://gpt-index.readthedocs.io/en/latest/reference/service_context.html

service_context = ServiceContext.from_defaults(chunk_size_limit=512)


In [7]:
# initialize simple vector indices + global vector index
# NOTE: don't run this cell if the indices are already loaded! 
# generates many calls to openai to compute embedding vectors
# https://gpt-index.readthedocs.io/en/latest/examples/vector_stores/SimpleIndexDemo.html

# index_set = {}

# for year in years:    
#     print(datetime.now(), 'indexing', year)
#     index_id = "index_%d" % year
#     cur_index = GPTVectorStoreIndex.from_documents(doc_set[year]
#                                                    service_context=service_context)
#     index_set[year] = cur_index
#     cur_index.storage_context.persist(index_id)


In [8]:
# load previously created indexes
index_set = {}
for year in years:
    index_id = "index_%d" % year
    print(datetime.now(), 'loading', year)
    # load index
    cur_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=index_id))
    index_set[year] = cur_index


2023-05-07 09:51:45.613594 loading 2022


INFO:llama_index.indices.loading:Loading all indices.


Loading all indices.
2023-05-07 09:51:45.923216 loading 2021


INFO:llama_index.indices.loading:Loading all indices.


Loading all indices.
2023-05-07 09:51:46.274689 loading 2020


INFO:llama_index.indices.loading:Loading all indices.


Loading all indices.
2023-05-07 09:51:46.615722 loading 2019


INFO:llama_index.indices.loading:Loading all indices.


Loading all indices.


In [9]:
# NOTE: this global index is a single vector store containing all documents
# Only relevant for the section below: "Can a single vector index answer questions across years?"
# this generates many calls, and should be possible to duplicate by composing global index from individual indexes

# global_index = GPTVectorStoreIndex.from_documents(all_docs,
#                                                   service_context=service_context)
# global_index.storage_context.persist("index_global")


In [10]:
global_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="index_global"))


INFO:llama_index.indices.loading:Loading all indices.


Loading all indices.


In [11]:
query_engine = index_set[2020].as_query_engine(retriever_mode="embedding", 
                                               service_context=service_context,                                     
                                               similarity_top_k=3,
                                               verbose=True,
                                              )
query = "What were some of the biggest risk factors in 2020?"
response = query_engine.query(query)
print(response)


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens


> [retrieve] Total embedding token usage: 11 tokens


Token indices sequence length is longer than the specified maximum sequence length for this model (1318 > 1024). Running this sequence through the model will result in indexing errors
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1502 tokens


> [get_response] Total LLM token usage: 1502 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens

The biggest risk factors in 2020 included:
- Changes in normal business practices necessitated by the outbreak and related governmental actions.
- Privacy, cybersecurity and fraud risks associated with increased remote working.
- Legal or regulatory challenges to our understanding of applicable legal and regulatory requirements.
- Heightened risks associated with the launch of new services, features, or health and safety requirements.
- Financial impacts of the COVID-19 pandemic, including reductions in workforce and changes to pricing models.
- Uncertainty around the ultimate impact of the pandemic on our future business operations, liquidity, financial condition, and results of operations.
- Potential fines or other enforcement measures resulting from legal or regulatory challenges.
- Adverse impacts on our business partners and third-party vendors.
- Extreme volatility in financial markets.
- Risk of Drivers being classified as

In [12]:
query = "What were some of the signifcant acquisitions?"
response = query_engine.query(query)
print(response)


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens


> [retrieve] Total embedding token usage: 11 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1371 tokens


> [get_response] Total LLM token usage: 1371 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens

Some of the significant acquisitions include the acquisition of Careem, the purchase of a controlling interest in Cornershop, the acquisition of Postmates, the acquisition of Routematch Holdings, Inc., and the acquisition of Cornershop Global LLC.


In [30]:
query_all = global_index.as_query_engine(retriever_mode="embedding", 
                                         service_context=service_context,   
                                         similarity_top_k=3,
#                                          response_mode="tree_summarize",
                                         verbose=True,
                                    )
risk_query_str = "What are some of the biggest risk factors in each year?"
response = query_all.query(risk_query_str)
print(str(response))


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 12 tokens


> [retrieve] Total embedding token usage: 12 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1700 tokens


...

> [get_response] Total embedding token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1700 tokens


> [get_response] Total LLM token usage: 1700 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens

2019:
- Drivers being classified as employees, workers or quasi-employees instead of independent contractors.
- High competition in the mobility, delivery, and logistics industries.
- Lowering of fares or service fees to remain competitive.
- Security breaches exposing the company to liability and increasing the risk of litigation and governmental investigation.

2020:
- Drivers being classified as employees, workers or quasi-employees instead of independent contractors.
- High competition in the mobility, delivery, and logistics industries.
- Lowering of fares or service fees to remain competitive.
- Security breaches exposing the company to liability and increasing the risk of litigation and governmental investigation.
- Inability to anticipate and prevent security techniques.
- Potential misappropriation of confidential, proprietary, or personal information.

2022:
- Drivers being classified as employees, workers or quasi-emplo

In [14]:
# create summary text for each doc
summaries = {}
for year in years:
    summaries[year] = f"UBER 10-k Filing for {year} fiscal year"
    

In [15]:
# set number of output tokens
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, max_tokens=512))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)


In [16]:
graph = ComposableGraph.from_indices(
    GPTListIndex,
    [index_set[y] for y in years],
    [summaries[y] for y in years],
    service_context=service_context
)


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens


> [build_index_from_nodes] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


> [build_index_from_nodes] Total embedding token usage: 0 tokens


In [38]:
custom_query_engines = {
    my_index.index_id: my_index.as_query_engine(
        similarity_top_k=1,
        response_mode="tree_summarize",
    )
    for my_index in [index_set[y] for y in years]
}

In [39]:
risk_query_str = (
    "Describe the current risk factors. If the year is provided in the information, "
    "provide that as well. If the context contains risk factors for multiple years, "
    "explicitly provide the following:\n"
    "- A description of the risk factors for each year\n"
    "- A summary of how these risk factors are changing across years"
)


In [40]:
query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)
response = query_engine.query(risk_query_str)
print(response)

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 60 tokens


> [retrieve] Total embedding token usage: 60 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 634 tokens


...

> [get_response] Total embedding token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1027 tokens


> [get_response] Total LLM token usage: 1027 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens

The current risk factors for 2021 include the impact of the COVID-19 pandemic on parts of our business, the potential for Drivers to be classified as employees, workers or quasi-employees instead of independent contractors, the highly competitive nature of the mobility, delivery, and logistics industries, and the need to lower fares or service fees and offer Driver incentives and consumer discounts and promotions in order to remain competitive in certain markets. We have also incurred significant losses since inception, including in the United States and other major markets. 

The risk factors for 2020 are outlined in Item 1A of the information provided. These risk factors include unresolved staff comments, properties, legal proceedings, and mine safety disclosures. 

The risk factors for 2019 include interest rate risk, investment risk, and foreign currency risk. Interest rate risk relates to the 2016 Term Loan Facility and 2018 

In [41]:
print(response)


The current risk factors for 2021 include the impact of the COVID-19 pandemic on parts of our business, the potential for Drivers to be classified as employees, workers or quasi-employees instead of independent contractors, the highly competitive nature of the mobility, delivery, and logistics industries, and the need to lower fares or service fees and offer Driver incentives and consumer discounts and promotions in order to remain competitive in certain markets. We have also incurred significant losses since inception, including in the United States and other major markets. 

The risk factors for 2020 are outlined in Item 1A of the information provided. These risk factors include unresolved staff comments, properties, legal proceedings, and mine safety disclosures. 

The risk factors for 2019 include interest rate risk, investment risk, and foreign currency risk. Interest rate risk relates to the 2016 Term Loan Facility and 2018 Term Loan Facility, which are floating rate notes and a

In [42]:
print(response_summary.get_formatted_sources()[:300])


> Source (Doc id: b5b1a2c0-3346-4fac-8eed-e5ee54b6c5bf): 
The current risk factors for 2022 include: 
- Drivers being classified as employees, workers or ...

> Source (Doc id: 6b090db1-3c38-4a41-bd88-24191a2753eb): 
The current risk factors for 2021 include the COVID-19 pandemic and the impact of a


In [43]:
response_tmp = index_set[2022].as_query_engine(retriever_mode="embedding", 
                                               service_context=service_context,                                     
                                               similarity_top_k=3,
                                               verbose=True,
                                              ).query(risk_query_str)
print(str(response_tmp))


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 60 tokens


> [retrieve] Total embedding token usage: 60 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1606 tokens


> [get_response] Total LLM token usage: 1606 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens

For the year 2022, the risk factors include: 

- Drivers being classified as employees, workers or quasi-employees instead of independent contractors.
- The mobility, delivery, and logistics industries being highly competitive, with well-established and low-cost alternatives that have been available for decades, low barriers to entry, low switching costs, and well-capitalized competitors in nearly every major geographic region.
- Lowering of fares or service fees, and offering of significant Driver incentives and consumer discounts and promotions in order to remain competitive in certain markets.
- Incurring significant losses since inception, including in the United States and other major markets.
- Attracting and maintaining a critical mass of Drivers, consumers, merchants, Shippers, and Carriers.
- Retaining and attracting high-quality personnel, and continued attrition, future attrition, or unsuccessful succession planning.
- 

In [46]:
response = global_index.as_query_engine(retriever_mode="embedding", 
                                        service_context=service_context,   
                                        similarity_top_k=4,
                                        verbose=True,
                                       ).query(risk_query_str)
print(str(response))


INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens


> [retrieve] Total LLM token usage: 0 tokens


INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 60 tokens


> [retrieve] Total embedding token usage: 60 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2107 tokens


> [get_response] Total LLM token usage: 2107 tokens


INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


> [get_response] Total embedding token usage: 0 tokens

Year 2020:
The risk factors include Drivers being classified as employees, workers or quasi-employees instead of independent contractors, the mobility, delivery, and logistics industries being highly competitive, and the need to lower fares or service fees and offer significant Driver incentives and consumer discounts and promotions to remain competitive.

Year 2021:
The risk factors include the COVID-19 pandemic and the impact of actions to mitigate the pandemic, Drivers being classified as employees, workers or quasi-employees instead of independent contractors, the mobility, delivery, and logistics industries being highly competitive, and the need to lower fares or service fees and offer significant Driver incentives and consumer discounts and promotions to remain competitive.

Year 2022:
The risk factors include Drivers being classified as employees, workers or quasi-employees instead of independent contractors, the mobility, 